In [ ]:
%%writefile kernel.cu
#include <iostream>
#include <cuda_runtime.h>
using namespace std;

__global__ void matrixMulKernel(float *A, float *B, float *C, int M, int N, int P, int offset) {
    int k = threadIdx.x + offset;

    float *a = A + k * M * N;
    float *b = B + k * N * P;
    float *c = C + k * M * P;

    for (int i = 0; i < M; i++) {
        for (int j = 0; j < N; j++) {
            for (int l = 0; l < P; l++) {
                c[i * P + l] += a[i * N + j] * b[j * P + l];
            }
        }
    }

    A = > sobai mile
    a => akhon kon matrix er kaj hobe

    1st matrix => A + 0 * M * N = A
    2nd matrix => A + 1 * M * N
    3rd matrix => A + 2 * M * N

    float* A =>  all matrix aksathe ache
    A[0] = > matrix[0]  er starting
    A[M*N] => matrix[1] starting

}

int main(int argc, char *argv[]) {
    if (argc < 3) {
        cout << "Usage: " << argv[0] << " <num_matrices> <max_threads>" << endl;
        return 1;
    }

    int K = stoi(argv[1]);  // number of matrices
    int x = stoi(argv[2]);  // number of threads


    int M = 400, N = 400, P = 400;
    int sizeA = K * M * N * sizeof(float); //how many byte in matrix A,  ( * K)
    int sizeB = K * N * P * sizeof(float);
    int sizeC = K * M * P * sizeof(float);

    float *h_A = new float[K * M * N];
    float *h_B = new float[K * N * P];
    float *h_C = new float[K * M * P];

    // random initialization
    for (int i = 0; i < K * M * N; i++) h_A[i] = rand();
    for (int i = 0; i < K * N * P; i++) h_B[i] = rand();

    //copy data to device
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, sizeA);
    cudaMalloc(&d_B, sizeB);
    cudaMalloc(&d_C, sizeC);

    cudaMemcpy(d_A, h_A, sizeA, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, sizeB, cudaMemcpyHostToDevice);

    matrixMulKernel<<<1, K>>>(d_A, d_B, d_C, M, N, P);
    cudaDeviceSynchronize();



    // launch kernel
    // We have to process batch by batch
    // as we may not have enough threads to process all matrices at once
    for (int i = 0; i < K; i += x) {
        int currentBatchSize = min(x, K - i);
        printf("Processing matrixs from %d to %d\n", i, i + currentBatchSize - 1);

    }

    //sync & copy
    cudaMemcpy(h_C, d_C, sizeC, cudaMemcpyDeviceToHost);

    // Output results
    /*
    for (int k = 0; k < K; k++) {
        cout << "Matrix C[" << k << "]: " << endl;
        for (int i = 0; i < M; i++) {
            for (int j = 0; j < P; j++) {
                cout << h_C[k * M * P + i * P + j] << " ";
            }
            cout << endl;
        }
        cout << endl;
    }
    */
    cout << "All multiplications completed successfully!" << endl;

    delete[] h_A;
    delete[] h_B;
    delete[] h_C;
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);

}

Writing kernel.cu


In [ ]:
!nvcc -arch=sm_75 kernel.cu -o kernel

In [ ]:
!time ./kernel 10 1

Processing matrixs from 0 to 0
Processing matrixs from 1 to 1
Processing matrixs from 2 to 2
Processing matrixs from 3 to 3
Processing matrixs from 4 to 4
Processing matrixs from 5 to 5
Processing matrixs from 6 to 6
Processing matrixs from 7 to 7
Processing matrixs from 8 to 8
Processing matrixs from 9 to 9
All multiplications completed successfully!

real	0m11.518s
user	0m11.110s
sys	0m0.181s


In [ ]:
!time ./kernel 10 2

Processing matrixs from 0 to 1
Processing matrixs from 2 to 3
Processing matrixs from 4 to 5
Processing matrixs from 6 to 7
Processing matrixs from 8 to 9
All multiplications completed successfully!

real	0m6.115s
user	0m5.462s
sys	0m0.169s


In [ ]:
!time ./kernel 10 3

Processing matrixs from 0 to 2
Processing matrixs from 3 to 5
Processing matrixs from 6 to 8
Processing matrixs from 9 to 9
All multiplications completed successfully!

real	0m4.921s
user	0m4.676s
sys	0m0.154s


In [ ]:
!time ./kernel 10 4

Processing matrixs from 0 to 3
Processing matrixs from 4 to 7
Processing matrixs from 8 to 9
All multiplications completed successfully!

real	0m3.817s
user	0m3.601s
sys	0m0.155s


In [ ]:
!time ./kernel 10 10

Processing matrixs from 0 to 9
All multiplications completed successfully!

real	0m1.885s
user	0m1.744s
sys	0m0.132s


In [ ]:
!time ./kernel 10 15

Processing matrixs from 0 to 9
All multiplications completed successfully!

real	0m1.878s
user	0m1.750s
sys	0m0.121s


In [ ]:
!time ./kernel 10 20 && sleep 2

Processing matrixs from 0 to 9
All multiplications completed successfully!

real	0m1.909s
user	0m1.742s
sys	0m0.148s
